In [1]:
import pandas as pd
import numpy as np
import re
import string

In [2]:
text = 'awsome product. I love it'

In [3]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text =text.replace(punctuation, '')
    return text


In [4]:
with open ('../static/model/corpora/stopwords/english', 'r') as file:
    sw=file.read().splitlines()

In [5]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

In [6]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data['tweet']=data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet']=data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','', x,flags=re.MULTILINE) for x in x.split()))
    data["tweet"]=data["tweet"].str.replace('/d+','', regex=True)
    data["tweet"]=data["tweet"].str.replace('/d+','', regex=True)
    data["tweet"]= data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"]=data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [7]:
text_cleaned = preprocessing(text)

In [8]:
text_cleaned

0    awsom product. love
Name: tweet, dtype: object

In [9]:
with open ('../static/model/vocabulary.txt', 'r') as vocabulary_file:
    vocabulary_text = vocabulary_file.read().splitlines()
  

In [10]:
len(vocabulary_text)

1192

In [11]:
import numpy as np

def vectorizer(ds, vocabulary):
    vectorizer_list = []
    vocab_set = set(vocabulary)  # Use a set for faster lookup
    
    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary), dtype=np.float16)  # Use np.float16 to reduce memory usage
        sentence_words = set(sentence.split())  # Split and convert sentence to a set to speed up membership checking
        
        for i, word in enumerate(vocabulary):
            if word in sentence_words:  # Check if word exists in the sentence
                sentence_list[i] = 1
                
        vectorizer_list.append(sentence_list)
    
    # Convert the list to a numpy array only once
    vectorizer_list_new = np.asarray(vectorizer_list, dtype=np.float16)
    
    return vectorizer_list_new


In [12]:
vectorized_test_data = vectorizer(text_cleaned,vocabulary_text)

In [13]:
vectorized_test_data

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [14]:
import pickle
with open ('../static/model/model.pkl', 'rb') as f:
    model1 = pickle.load(f)
    model1.predict(vectorized_test_data)

In [16]:
predictions=  model1.predict(vectorized_test_data)
if predictions[0]==0:
    print("Positive")
else:
    print("Negative")

Positive
